In [12]:
import torch
import datasets
import torchvision
import torch.distributed as dist



In [13]:
origin_dataset = datasets.load_dataset('imagenet', '/home/data/Imagenet/', 'pair', batch_size=4)
custom_dataset = datasets.CustomDataset('/home/data/Imagenet')

In [14]:
custom_dataset = torch.utils.data.DataLoader(custom_dataset,batch_size=4,shuffle=True)

In [16]:
next(iter(origin_dataset[0]))[0].shape

torch.Size([8, 3, 224, 224])

In [17]:
next(iter(custom_dataset))[0].shape

torch.Size([4, 3, 224, 224])

TypeError: 'DataLoader' object is not subscriptable

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

input_size = 3
output_size = 2

batch_size = 30
data_size = 100

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return self.len


rand_loader = DataLoader(dataset=RandomDataset(input_size,data_size), batch_size=batch_size, shuffle=True)

In [18]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("\t In Model : input size", input.size(), "output size", output.size())
        return output

In [19]:
model = Model(input_size, output_size)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    
    model = nn.DataParallel(model)

model.to(device)

Let's use 8 GPUs!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=3, out_features=2, bias=True)
  )
)

In [20]:
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside: input size", input.size(),
          "output_size", output.size())

	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([2, 3]) output size torch.Size([2, 2])
Outside: input size torch.Size([30, 3]) output_size torch.Size([30, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3]) output size torch.Size([4, 2])
	 In Model : input size torch.Size([4, 3